# Regression

Just like classification, we will use previous data to predict future observations. However, instead of predicting cateogrical variables, we will be predicting numerical variables.

## Regression with $K$-nearest neighbours

```slice_sample(dataset, n = some_number)``` - takes a small random sample from a dataset.

***Let's say we want to predict the price of a house that is 2000 square feet large.***

<img src="media/2000_square_feet_house.png" width="300px">

We will use the neighboring points to the find the new point of interest to suggest/predict what the sales price might be.

```r
sacramento <- read_csv(...)
small_sacramento <- slice_sample(sacramento, n = 30)

nearest_neighbors <- small_sacramento %>%
                     mutate(diff = abs(2000 - sqft)) %>%
                     arrange(diff) %>%
                     slice(1:5) # subset the first 5 rows
```

<img src="media/nearest_neighbors.png" width="400px">

Then we can predict the mean of the nearest neighbors.

```r
prediction <- nearest_neighbors %>%
              summarise(predicted = mean(price))

prediction

> 326324
```

<img src="media/regression_prediction.png" width="300px">

***NOTE: The best part about KNN regression is that it also works well with non-linear relationships because the algorithm has very few assumptions about what the data must look like.***

### Training, evaluating and tuning the model

In KNN classification, we used accuracy to see how our predictions matched the true labels. In KNN regression, we will use root mean square prediction error (RMSPE) instead because our predictions will never match the true response variable values.

***NOTE:***
- When predicting and evaluating the prediction quality on the ***TRAINING DATA*** we say $RMSE$.
    - measures how well the model predicts on data it was trained with.
- When predicting and evaluating the prediction quality on the ***TESTING DATA*** we say $RMSPE$.
    - measures the prediction quality (predicts on data it was not trained with)

$ RMSPE = \sqrt{\frac{1}{n} \sum_{i = 1} ^ n (y_i - \hat{y}_i)^2} $

Where:
- $n$ is the number of observations
- $y_i$ is the observed value for the $i^th$ observation
- $\hat{y}_i$ is the forecasted/predicted value for the $i^th$ observation

<img src="media/rmspe.png" width="300px">

#### Finding the Optimal $K$

```r
some_recipe <- recipe(label ~ predictors, data = dataset_training) %>%
               step_scale(all_predictors()) %>%
               step_center(all_predictors())
               
some_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
             set_engine("kknn") %>%
             set_mode("regression")

some_vfold <- vfold_cv(dataset_training, v = 5, strata = label)

some_workflow <- workflow() %>%
                 add_recipe(some_recipe) %>%
                 add_model(some_spec)

gridvals <- tibble(neighbors = seq(from = 1, to = 200, by = 3))

some_results <- some_workflow %>%
                tune_grid(resamples = some_vfold, grid = gridvals) %>%
                collect_metrics() %>%
                filter(.metric == "rmse")
```

<img src="media/optimal_k_regression.png" width="400px">

             

To find the most optimal number of neighbors $K$, we take the ***minimum*** RMSPE.

```r
some_min <- some_resuts %>%
            filter(mean == min(mean)))
```

### Evaluating on the test set

```r
kmin <- some_min %>% pull(neighbors)

some_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = kmin) |>
             set_engine("kknn") |>
             set_mode("regression")

some_fit <- workflow() |>
            add_recipe(some_recipe) |>
            add_model(some_spec) |>
            fit(data = some_training)

some_summary <- some_fit |>
                predict(some_test) |>
                bind_cols(some_test) |>
                metrics(truth = label, estimate = .pred) |>
                filter(.metric == 'rmse')
```

<img src="media/evaluating_on_test_set.png" width="400px">

Our final model's test error assessed by RMPSPE is $89279.

To show the predictions that our final model makes across the range of houses:

```r
some_preds <- tibble(sqft = seq(from = 500, to = 5000, by = 10))

some_preds <- some_fit %>%
              predict(some_preds) %>%
              bind_cols(some_preds)

plot_final <- ggplot(some_train, aes(x = car1, y = var2)) +
                  geom_point(alpha = 0.4) +
                  geom_line(data = some_preds, 
                            mapping = aes(x = var1, y = .pred), 
                            color = "blue") +
                  xlab("House size (square feet)") +
                  ylab("Price (USD)") +
                  scale_y_continuous(labels = dollar_format()) +
                  ggtitle(paste0("K = ", kmin)) + 
                  theme(text = element_text(size = 12))
```
***NOTE THE ```geom_line```***

<img src="media/final_model_plot.png" width="350px">

### Multivariable KNN Regression

We can use the predictor selection algorithm from classification to select the best predictors in this case too.

First we'll build a new model specification and recipe for the analysis.

```r
some_recipe <- recipe(label ~ predictors, data = some_train) |>
               step_scale(all_predictors()) |>
               step_center(all_predictors())

some_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
             set_engine("kknn") |>
             set_mode("regression")
```

Next, we'll use 5-fold cross-validation to choose the number of neighbors vai the minimum RMSPE:

```r
gridvals <- tibble(neighbors = seq(1, 200))

some_workflow <- workflow() |>
                 add_recipe(some_recipe) |>
                 add_model(some_spec) |>
                 tune_grid(some_vfold, grid = gridvals) |>
                 collect_metrics() |>
                 filter(.metric == "rmse") |>
                 filter(mean == min(mean))

some_k <- some_workflow |> pull(neighbors)
```

Then, we need to re-train the model on the entire training data set.

```r
some_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = some_k) |>
             set_engine("kknn") |>
             set_mode("regression")

knn_fit <- workflow() |>
           add_recipe(some_recipe) |>
           add_model(some_spec) |>
           fit(data = some_train)

knn_preds <- knn_fit |>
             predict(some_test) |>
             bind_cols(some_test)

knn_mets <- metrics(knn_preds, truth = label, estimate = .pred) %>%
            filter(.metric == 'rmse')
```

### Summary

**Strengths**: $K$-nearest neighbors regression:
1. is a simple, intuitive algorithm
2. requires few assumptions about what the data must look like
3. works well with non-linear relationships (i.e., if the relationship is not a straight line).

**Weaknesses**: $K$-nearest neighbors regression:
1. becomes very slow as the training data gets larger
2. may not perform well with a large number of predictors
3. may not predict well beyond the range of values input in your training data.

## Linear Regression

Again, with the house price example, if we wanted to predict the price (USD) of a house that is 2000 square feet:

$house \space sale \space price = \beta_0 + \beta_1 \times (house \space size)$

where:
- $\beta_0$ is the vertical intercept of the line (the price when house size is 0)
- $\beta_1$ is the slope fo the line (how quickly the price increases as you increase house size)

<img src="media/linear_regression_house_price.png" width="300px">

Simple linear regression chooses the stright line of best fit by choosing the line that minimizes the **average squared vertical distance** between itself and each of the observed data points in training data.

<img src="media/linear_regression_choosing_line.png" width="300px">

Finally, to assess the predictive accuracy of the observed data points, we use RMSPE.

***NOTE: RMSPE is the RMSE of the TEST DATA.***

### Performing Linear Regression

Load the library functions, get the data and split the data.

```r
library(tidyverse)
library(tidymodels)

set.seed(1234)

sacramento <- read_csv("data/sacramento.csv")

sacramento_split <- initial_split(sacramento, prop = 0.6, strata = price)
sacramento_train <- training(sacramento_split)
sacramento_test <- testing(sacramento_split)
```

Then we create model specifications, recipe, and workflow.

```r
lm_spec <- linear_reg() |>
           set_engine("lm") |>
           set_mode("regression")

lm_recipe <- recipe(price ~ sqft, data = sacramento_train)

lm_fit <- workflow() |>
          add_recipe(lm_recipe) |>
          add_model(lm_spec) |>
          fit(data = sacramento_train)
```

***NOTE: We do not standardize our predictors! In leinear regression, standardization does not affect the fit (it does effective the coefficients in the equation)***

<img src="media/fit_linear_regression.png" width="400px;">

Our coefficients are (intercept) $\beta_0 = 12292$ and (slope) $\beta_1 = 140$. This means that the equation of best fit is:

$house \space sale \space price = 12292 + 140 \times (house \space size)$

Next, we predict on the test data to assess how well our model does:

```r
lm_test_results <- lm_fit |>
                   predict(sacramento_test) |>
                   bind_cols(sacramento_test) |>
                   metrics(truth = price, estimate = .pred)
```

<img src="media/model_test_error_linear_regression.png" width="400px">

Our final model's test error as assessed by RMSPE is 82,342 (remember that this is in units of the target/response variable, in this case USD).

Next, we can plot our linear regression line using ```geom_smooth```:

```r
lm_plot_final <- ggplot(sacramento_train, aes(x = sqft, y = price)) +
                     geom_point(alpha = 0.4) +
                     xlab("House size (square feet)") +
                     ylab("Price (USD)") +
                     scale_y_continuous(labels = dollar_format()) +
                     geom_smooth(method = "lm", se = FALSE) + 
                     theme(text = element_text(size = 12))
```

<img src="media/lm_plot_final.png" width="300px">


We can, then, extract the coefficients from our model by accesssing the fit object that is output. We apply the ```tidy``` function to convert the result into a dataframe:

```r
coeffs <- lm_fit |>
          pull_workflow_fit() |>
          tidy()
```

<img src="media/linear_reg_coeff.png" width="500px">

### Comparing Linear Regression with KNN Regression

<img src="media/lin_reg_vs_knn.png" width="400px">

There is a major interprebility advantage in limiitng the model to the straight line. However, there are disadvantages, particularly when the relationship between the target and predictor is NOT linear but some other shape. In these cases the predcition model from LinReg would underfit (have high bias) meaning that model/predicted values do not match the actual observed values well.

**Extrapolation**: Predicting outside the range of the observed data.

Note that KNN regression becomes "flat" at the left and right boundaries of the data, while the linear model predicts a constant slope. Depending on the application, the flat or constant slope trend may make more sense.

### Multivariable Linear Regression

Like KNN regression, we just add more predictors to the model; however, we do not need to use cross-validation to choose any parameters, nor do we need to standardize (i.e., center and scale) the data for linear regression.

```r
mlm_recipe <- recipe(price ~ sqft + beds, data = sacramento_train)

mlm_fit <- workflow() |>
           add_recipe(mlm_recipe) |>
           add_model(lm_spec) |>
           fit(data = sacramento_train)

lm_mult_test_results <- mlm_fit |>
                        predict(sacramento_test) |>
                        bind_cols(sacramento_test) |>
                        metrics(truth = price, estimate = .pred)

mcoeffs <- mlm_fit |>
           pull_workflow_fit() |>
           tidy()
```

<img src="media/multi_coefficients.png" width="400px">
<img src="media/multi_test_results.png" width="400px">

And then we can use the coefficiencts to write a mathematical equation to descrbe the prediction plane:

$house \space sale \space price = \beta_0 + \beta_1 \times (house \space size) + \beta_2 \times (number \space of \space bedrooms)$

In this case:

$house \space sale \space price = 63475 + 166 \times (house \space size) - 28761 \times (number \space of \space bedrooms)$

And reading the predictor error, we find that our RMSPE is 81,417.89

### Multicollinearity and Outliers

There are two common issues with linear regression: **1. Outliers** and **2. Collinear Predictors**

#### Outliers

Outliers can have too much influnce on the line of best fit. For instance, given our previous graph, adding one single oultier (red dot) can cause the line to shift tremendously:

<img src="media/lin_reg_outliers.png" width="300px">

This can be fixed with enough data. As long as you have enough data, the inlcusion of one or two outliers will typically not have a large effect on the line of best fit.

<img src="media/lin_reg_outliers_lots_data.png" width="300px">


#### Multicollinearity

If you include multiple predictors that are strongly linearly related to one another, the coefficients that are described in the plane of best fit can be very unreliable. Small changes to the data can result in large change in the coefficients.

<img src="media/multicollinearity.png" width="300px">

If we fit the multivariable inear regression model on this data, then the plane of best fit has regresion coefficients that are very sensitive to the exact values in the data.

Best Fit 1: $house \space sale \space price = 3682 + (-43) \times (house \space size \space 1 \space (ft^2)) + (182) \times (house \space size \space 2 \space (ft^2))$ <br>
Best Fit 2: $house \space sale \space price = 20596 + (312) \times (house \space size \space 1 \space (ft^2)) + (-172) \times (house \space size \space 2 \space (ft^2))$ <br>
Best Fit 3: $house \space sale \space price = 6673 + (37) \times (house \space size \space 1 \space (ft^2)) + (104) \times (house \space size \space 2 \space (ft^2))$

### Designing New Predictors

Sometimes our predictor variable will not have a nearly linear relationship with our response variable. In cases like these, the only option is to obtain measurements of more useful variables. We can do this by scaling the variables.

<img src="media/nonlinear_relationship_predictors.png" width="300px">

We can create a new predictor variable by scaling the previous ones:

```r
df <- df %>%
      mutate(z = x^3)
```

<img src="media/new_predictors.png" width="300px">

## GGPairs

Creates scatterplot of all the columns we are interested in including our model.

```r
credit_eda <- credit_training %>%
              ggpairs(columns = 1:3)
```

<img src="media/ggpairs.png" width="400px">